# AOPC_Project_County_Data

## Libraries

In [1]:
import pandas as pd
import numpy as np
import random 
import re
import requests
from bs4 import BeautifulSoup

## Auxillary Functions

Data by Excel Sheet Name:

In [35]:
def data(sheet):
    return pd.read_excel("AOPC_Project_MetaData.xlsx",sheet_name=f"{sheet}")

Wider Notebook:

In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

C:\Users\rcbre\AppData\Local\Temp\ipykernel_13628\3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


## Reference_Tables

### PA_Counties

In [4]:
PA_Counties = data("PA_Counties")
Population_Total = PA_Counties.Population.sum()
PA_Counties['Population_%'] = PA_Counties['Population']/Population_Total
PA_Counties['10000'] = (PA_Counties['Population_%'] * 10000).round()
b = PA_Counties['10000'].map(int)
PA_Counties['1000000'] = (PA_Counties['Population_%'] * 1000000).round()
c = PA_Counties['1000000'].map(int)
PA_Counties['5000000'] = (PA_Counties['Population_%'] * 5000000).round()
d = PA_Counties['5000000'].map(int)
PA_Counties['500000'] = (PA_Counties['Population_%'] * 500000).round()
e = PA_Counties['500000'].map(int)
ids = PA_Counties.Location_ID.map(str)
grand = []
milli = []
half = []
five = []
# Create location foreign keys for employees and cases respectively
for i,j in zip(b,ids):
    grand.extend([j]*i)
for i,j in zip(c,ids):
    milli.extend([j]*i)
for i,j in zip(d,ids):
    five.extend([j]*i)
for i,j in zip(e,ids):
    half.extend([j]*i)
PA_Counties

,Location_ID,County,County_Seat,State,Population,Population_%,10000,1000000,5000000,500000
0,1,Adams County,Gettysburg,Pennsylvania,101407,0.007983,80.0,7983.0,39917.0,3992.0
1,2,Allegheny County,Pittsburgh,Pennsylvania,1223348,0.096309,963.0,96309.0,481543.0,48154.0
2,3,Armstrong County,Kittanning,Pennsylvania,68941,0.005427,54.0,5427.0,27137.0,2714.0
3,4,Beaver County,Beaver,Pennsylvania,170539,0.013426,134.0,13426.0,67129.0,6713.0
4,5,Bedford County,Bedford,Pennsylvania,49762,0.003918,39.0,3918.0,19588.0,1959.0
...,...,...,...,...,...,...,...,...,...,...
62,63,Washington County,Washington,Pennsylvania,207820,0.016361,164.0,16361.0,81804.0,8180.0
63,64,Wayne County,Honesdale,Pennsylvania,52822,0.004158,42.0,4158.0,20792.0,2079.0
64,65,Westmoreland County,Greensburg,Pennsylvania,365169,0.028748,287.0,28748.0,143740.0,14374.0
65,66,Wyoming County,Tunkhannock,Pennsylvania,28276,0.002226,22.0,2226.0,11130.0,1113.0


### US_Census_Bureau_Demographics
SOURCE: https://www.census.gov/quickfacts/fact/table/

#### Extract and Transform in One Step

Auxillary Function to Clean Characters:

In [5]:
def clean(x):
    x = re.sub("\ue840\ue83f", "", x)
    x = re.sub(" ", "", x)
    x = re.sub("%", "", x)
    return x

Replace Z represented to by Zero with '0'

In [6]:
def Z(x):
    if x == 'Z':
        return 0
    else:
        return x

- Below, we extract the html from each of the US Census pages representing data for each Pennsylvania county.
- Each of the html files are converted to text files that are then cleaned and transformed to obtain a county name with percentages pertaining to racial demographics

In [7]:
URLs=[]
for county in PA_Counties.County:
    URLs.append(f'''https://www.census.gov/quickfacts/fact/table/{county.lower().replace(' ','')}pennsylvania/PST045222''')
TEXTS=[]
for url in URLs:
    TEXTS.append(requests.get(url).content.decode('utf-8'))
TABLES=[]
for text in TEXTS:
    TABLES.append(pd.read_html(text))
Race = {}
Gender = {}
Index = TABLES[1][1]['Population'].iloc[15:22]
Index2 = TABLES[1][1]['Population'].iloc[12:13]
for table,county in zip(TABLES,PA_Counties.County):
    Race[f'{county}'] = table[1]['Unnamed: 1'].iloc[15:22].tolist()
    Gender[f'{county}'] = table[1]['Unnamed: 1'].iloc[12:13].tolist()
RACE = pd.DataFrame(Race, index=Index)
GENDER = pd.DataFrame(Gender, index=Index2)
RACE = RACE.applymap(clean)
RACE = RACE.applymap(Z)
GENDER = GENDER.applymap(clean)
# rename column names to simplify
change = {'Black or African American alone, percent(a)':'Black or African American',
'American Indian and Alaska Native alone, percent(a)': 'Native American',
'Asian alone, percent(a)': 'Asian',
'Native Hawaiian and Other Pacific Islander alone, percent(a)': 'Pacific Islander',
'Two or More Races, percent': 'Two or More Races',
'Hispanic or Latino, percent(b)': 'Hispanic or Latino',
'White alone, not Hispanic or Latino, percent': 'White, not Hispanic or Latino'}
# drop the two or more races category
RACE = RACE.drop(index=['Two or More Races, percent'])
RACE = RACE.rename(index=change)
RACE.to_csv('RACE.csv',index = False)
GENDER.to_csv('GENDER.csv',index = False)
RACE

,Adams County,Allegheny County,Armstrong County,Beaver County,Bedford County,Berks County,Blair County,Bradford County,Bucks County,Butler County,...,Susquehanna County,Tioga County,Union County,Venango County,Warren County,Washington County,Wayne County,Westmoreland County,Wyoming County,York County
Population,,,,,,,,,,,,,,,,,,,,,
Black or African American,2.1,13.5,1.0,6.6,0.7,7.9,2.2,0.9,4.7,1.5,...,0.8,0.8,7.5,1.3,0.6,3.4,3.7,2.8,1.2,7.5
Native American,0.4,0.2,0.2,0.2,0.2,1.0,0.2,0.3,0.3,0.2,...,0.3,0.3,0.6,0.2,0.3,0.2,0.4,0.2,0.3,0.4
Asian,0.9,4.4,0.3,0.6,0.5,1.6,0.7,0.7,5.5,1.6,...,0.4,0.6,1.6,0.5,0.5,1.2,0.8,1.0,0.5,1.6
Pacific Islander,0.1,0,0,0,0.1,0.2,0,0,0.1,0,...,0.1,0,0.1,0,0.1,0,0,0,0,0.1
Hispanic or Latino,7.4,2.4,0.9,2.0,1.3,23.9,1.4,1.7,6.1,1.8,...,2.0,1.4,6.3,1.2,1.3,1.9,5.2,1.4,2.3,8.9
"White, not Hispanic or Latino",88.4,77.5,96.7,88.5,96.5,68.7,94.1,95.4,82.4,94.0,...,95.7,96.0,84.1,95.6,96.4,91.5,89.3,93.2,94.7,81.4


## Human_Resources

### Employee_Surveys
- Here we insert values ranging from 1 (strongly disagree) to 10 (strongly)
- We have created a story line for the dashboard where workers want childcare at the work place so we will make sure we strongly agree with questions 18,19 below

In [8]:
Employee_Surveys=data("Employee_Surveys")
Employee_Surveys.columns = Employee_Surveys.loc[0,:]
Employee_Surveys.drop([0],axis=0,inplace=True)
#Employee_Surveys.columns[18]
#Employee_Surveys.columns[19]
Questions = Employee_Surveys.columns[1:28].to_list()
ten = [*range(1,11)]
Employee_Survey = pd.DataFrame()
Employee_Survey['Employee_id'] = [i for i in range(1,10001)]
for i in range(len(Questions)):
    if i not in [18,19]:
        x = random.randint(6,10)
        y = random.randint(7,10)
        weights = [1,1,2,2,4,x,x,y,6,6]
        results = random.choices(ten,weights=weights, k = 10000)
        Employee_Survey[f'{Questions[i]}'] = results
    if i in [18,19]:
        x = random.randint(6,10)
        y = random.randint(7,10)
        weights = [1,1,2,2,4,x,x,y,30,20]
        results = random.choices(ten,weights=weights, k = 10000)
        Employee_Survey[f'{Questions[i]}'] = results

Employee_Survey.to_csv('Employee_Survey.csv',index = False)

In [9]:
Employee_Survey

,Employee_id,My pay is fair.,My salary is the most important factor in rating this workplace.,"In the past 3 years, I have received a pay increase.","Prior to starting work/project, my work is clearly defined to me.",I feel overwhelmed by the amount of work I have to do per a week.,The work I perform is relevant to my position.,"On average, I feel overworked.",I feel my work adds value to my workplace.,I feel like I belong at my workplace.,...,"In a typical work week, I work my set hours.","In past year, I was unable to work due to childcare.",I feel that a workplace childcare center would beneft my work life.,There is a health work life balance at my job.,I feel the leadershipt at my working location are very competent.,I am fairly evaluated for my performance evaluation.,I feel respected by my peers and supervisor.,I would recommend this workplace to others.,I enjoy working at this workplace.,There are other organizations that I would rather work at.
0,1,9,8,9,9,6,6,6,5,6,...,3,5,1,7,7,4,7,7,8,7
1,2,8,8,6,3,4,8,5,7,8,...,3,8,9,9,6,3,9,10,8,7
2,3,6,6,8,10,6,6,7,9,10,...,10,4,7,10,7,7,5,4,9,2
3,4,7,3,9,6,8,9,10,8,9,...,2,8,7,5,6,6,8,6,4,3
4,5,9,6,6,10,4,8,7,8,7,...,10,10,1,7,7,6,5,8,3,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,1,10,6,9,10,9,7,7,7,...,6,10,9,9,3,6,10,5,7,4
9996,9997,7,8,10,10,5,8,7,5,8,...,6,3,9,10,9,7,5,9,5,10
9997,9998,5,6,7,9,6,1,8,8,8,...,5,10,9,6,7,8,8,6,1,7
9998,9999,6,10,7,10,9,8,5,6,6,...,10,9,7,10,10,4,7,9,6,7


### Employee_Data
- We have 10000 employees that are scattered throughout each county courthouse proportional to the population in each county
- Since, each column has unique data types and characteristics we must create each column separately

#### Auxillary Function to save space

In [10]:
def r(x,y):
    return random.randint(x,y)
def n(x,y):
    return np.random.normal(x,y)
def round_(x):
    return round(x,2)  #########

#### Employee_Data

In [11]:
Employee_Data = pd.DataFrame()
Employee_Data['Employee_id'] = [i for i in range(1,10001)]
Employee_Data['Age'] = [r(21,65) for i in range(10000)]
# add four 51 to Philadelphia county
Employee_Data['Location_ID'] = grand + ['51']*4
races = ['Black or African American','Native American','Asian','Pacific Islander',
'Hispanic or Latino','White, not Hispanic or Latino']
Employee_Data['Race'] = random.choices(races,weights=[8,1,2,0.5,5.5,83], k = 10000)
Employee_Data['Gender'] = random.choices(['M','F'],weights=[49,51], k = 10000)
jobs = ['Judge','Clerk','Attorney','Paralegal','Baliff','Janitor','HR','Researcher']
Employee_Data['Position'] = random.choices(jobs,weights=[1,5,2,4,1,1,1,1], k = 10000)
Employee_Data['Salary_2018'] = Employee_Data['Position'].map({'Judge': r(120000,200000),'Clerk': r(45000,55000),
                                                         'Attorney': r(100000,140000),'Paralegal': r(40000,60000),
                                                         'Baliff':r(40000,50000),'Janitor': r(28000,38800),
                                                         'HR': r(50000,60000),'Researcher':r(60000,70000)})
Employee_Data['Salary_2019'] = Employee_Data['Salary_2018'] * 1.05
Employee_Data['Salary_2020'] = Employee_Data['Salary_2019'] * 1.01
Employee_Data['Salary_2021'] = Employee_Data['Salary_2020'] * 1.01
Employee_Data['Salary_2022'] = Employee_Data['Salary_2021'] * 1.05
Employee_Data['Hire_Date'] = [pd.to_datetime(f'{r(1,12)}/{r(1,28)}/{r(1984,2020)}') for i in range(10000)]
Employee_Data['Termination_Date'] = [np.nan]*10000
Employee_Data['Work_Hours_YTD'] = np.random.normal(40,0.5,10000)
Employee_Data['Recruitment_Hours'] = np.random.normal(5,0.5,10000)
Employee_Data['Recruitment_Bonus'] = random.choices([5000,1000,0],weights=[2,6,2], k = 10000)
Employee_Data['Internal_Hire'] = random.choices(['Yes','No'],weights=[8,2], k = 10000)

,Employee_id,Age,Location_ID,Race,Gender,Position,Salary_2018,Salary_2019,Salary_2020,Salary_2021,Salary_2022,Hire_Date,Termination_Date,Work_Hours_YTD,Recruitment_Hours,Recruitment_Bonus,Internal_Hire
0,1,59,1,"White, not Hispanic or Latino",F,Attorney,116867,122710.35,123937.4535,125176.828035,131435.669437,1999-10-26,NaN,39.803058,5.153904,5000,No
1,2,39,1,"White, not Hispanic or Latino",M,Clerk,50968,53516.40,54051.5640,54592.079640,57321.683622,1991-12-17,NaN,39.518588,5.978117,0,Yes
2,3,48,1,"White, not Hispanic or Latino",F,Attorney,116867,122710.35,123937.4535,125176.828035,131435.669437,2013-10-05,NaN,39.181902,4.649308,1000,No
3,4,30,1,Hispanic or Latino,F,Clerk,50968,53516.40,54051.5640,54592.079640,57321.683622,1990-02-19,NaN,40.741572,5.434010,5000,Yes
4,5,24,1,Black or African American,M,Attorney,116867,122710.35,123937.4535,125176.828035,131435.669437,2017-03-28,NaN,40.351313,4.766877,0,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,62,67,"White, not Hispanic or Latino",M,Paralegal,47714,50099.70,50600.6970,51106.703970,53662.039169,1995-04-17,NaN,40.232884,4.605360,1000,No
9996,9997,47,51,"White, not Hispanic or Latino",F,Clerk,50968,53516.40,54051.5640,54592.079640,57321.683622,2014-07-03,NaN,39.924047,4.276861,0,No
9997,9998,59,51,"White, not Hispanic or Latino",M,Paralegal,47714,50099.70,50600.6970,51106.703970,53662.039169,2019-12-23,NaN,39.811971,5.172884,1000,Yes
9998,9999,43,51,"White, not Hispanic or Latino",F,Researcher,60386,63405.30,64039.3530,64679.746530,67913.733857,1994-05-19,NaN,39.544303,4.935532,1000,No


Data Tweak:
- Create a wider distribution for the Male Salary 

In [27]:
Employee_Data[Employee_Data['Gender'] =='M']['Salary_2022'][:5000]=Employee_Data[Employee_Data['Gender']=='M']['Salary_2022'][:5000]*1.2
Employee_Data.to_csv('Employee_Data.csv',index = False)
Employee_Data.head()

C:\Users\rcbre\AppData\Local\Temp\ipykernel_13628\4127016740.py:1: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  Employee_Data[Employee_Data['Gender'] =='M']['Salary_2022'][:5000]=Employee_Data[Employee_Data['Gender']=='M']['Salary_2022'][:5000]*1.2
C:\Users\rcbre\AppData\Local\Temp\ipykernel_13628\4127016740.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Employee_Data[Employee_Data['Gender'] =='M']['Salary_2022'][:5000]=Employee_Data[Employee_Data['Gender']=='M']['Salary_2022'][:5000]*1.2


,Employee_id,Age,Location_ID,Race,Gender,Position,Salary_2018,Salary_2019,Salary_2020,Salary_2021,Salary_2022,Hire_Date,Termination_Date,Work_Hours_YTD,Recruitment_Hours,Recruitment_Bonus,Internal_Hire
0,1,59,1,"White, not Hispanic or Latino",F,Attorney,116867,122710.35,123937.4535,125176.828035,131435.669437,1999-10-26,NaN,39.803058,5.153904,5000,No
1,2,39,1,"White, not Hispanic or Latino",M,Clerk,50968,53516.40,54051.5640,54592.079640,57321.683622,1991-12-17,NaN,39.518588,5.978117,0,Yes
2,3,48,1,"White, not Hispanic or Latino",F,Attorney,116867,122710.35,123937.4535,125176.828035,131435.669437,2013-10-05,NaN,39.181902,4.649308,1000,No
3,4,30,1,Hispanic or Latino,F,Clerk,50968,53516.40,54051.5640,54592.079640,57321.683622,1990-02-19,NaN,40.741572,5.434010,5000,Yes
4,5,24,1,Black or African American,M,Attorney,116867,122710.35,123937.4535,125176.828035,131435.669437,2017-03-28,NaN,40.351313,4.766877,0,Yes


#### Setting Up a Cross-Tab Table for Potential Chi-Square Analysis

In [28]:
def convert(x):
    if isinstance(x, float):
        return float(x)
    else:
        return str(x)

#### Zero values are converted to nearly zero to avoid error in Chi Square Contigency Test

In [29]:
def zero(x):
    if int(x) == 0:
        return 0.0000001
    else:
        return x

#### Racial Proportionss of Each PA County (according to U.S. Census)

In [30]:
RACE_T = RACE.T
RACE_T = RACE_T.reset_index().rename(columns={'index': 'County'})
Counties = PA_Counties.loc[:,['Location_ID', 'County']]
Counties_RACE = Counties.merge(RACE_T)
locations = Counties_RACE['Location_ID']
Counties_RACE = Counties_RACE.loc[:,['Black or African American','Native American','Asian','Pacific Islander',
'Hispanic or Latino','White, not Hispanic or Latino']].applymap(float)
Counties_RACE['Location_ID'] = locations
Counties_RACE = Counties_RACE.applymap(zero)
Counties_RACE.to_csv('Counties_RACE.csv',index=False)
Counties_RACE

,Black or African American,Native American,Asian,Pacific Islander,Hispanic or Latino,"White, not Hispanic or Latino",Location_ID
0,2.100000e+00,1.000000e-07,1.000000e-07,1.000000e-07,7.400000e+00,88.4,1
1,1.350000e+01,1.000000e-07,4.400000e+00,1.000000e-07,2.400000e+00,77.5,2
2,1.000000e+00,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,96.7,3
3,6.600000e+00,1.000000e-07,1.000000e-07,1.000000e-07,2.000000e+00,88.5,4
4,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.300000e+00,96.5,5
...,...,...,...,...,...,...,...
62,3.400000e+00,1.000000e-07,1.200000e+00,1.000000e-07,1.900000e+00,91.5,63
63,3.700000e+00,1.000000e-07,1.000000e-07,1.000000e-07,5.200000e+00,89.3,64
64,2.800000e+00,1.000000e-07,1.000000e+00,1.000000e-07,1.400000e+00,93.2,65
65,1.200000e+00,1.000000e-07,1.000000e-07,1.000000e-07,2.300000e+00,94.7,66


#### Cleanup Employee Data  - Must match 'Counties_RACE' Table Above

In [31]:
Employee_Race = Employee_Data.loc[:,['Location_ID','Race']]
Employee_Race = Employee_Race.groupby(['Location_ID'])['Race'].value_counts()
#Employee_Race = Employee_Race.reset_index().rename(index={'index':'Location_ID'})
Employee_Race = Employee_Race.to_frame()
Employee_Race = Employee_Race.rename(columns={'Race':'Count'})
Employee_Race = Employee_Race.reset_index()
Employee_Race = Employee_Race.pivot(index='Location_ID', columns='Race', values='Count')
Employee_Race = Employee_Race.reset_index().rename(index={'Race':'index'})
Employee_Race = Employee_Race.fillna(0)
Employee_Race['Total'] = Employee_Race.drop(['Location_ID'], axis=1).sum(axis=1)
for columns in Employee_Race.columns[1:-1]:
    Employee_Race[f'{columns}'] = np.multiply(np.divide(Employee_Race[f'{columns}'],Employee_Race['Total']), 100)
Employee_Race = Employee_Race.drop(['Total'], axis=1)
Employee_Race.Location_ID = Employee_Race.Location_ID.apply(int)
Employee_Race = Employee_Race.sort_values('Location_ID')
Employee_Race.index = Employee_Race.index.sort_values()
Employee_Race = Employee_Race.applymap(zero)
Employee_Race.to_csv('Employee_Race.csv',index=False)
Employee_Race

Race,Location_ID,Asian,Black or African American,Hispanic or Latino,Native American,Pacific Islander,"White, not Hispanic or Latino"
0,1,1.250000e+00,6.250000,3.750000e+00,1.250000e+00,1.000000e-07,87.500000
1,2,2.284528e+00,8.722741,6.230530e+00,1.973001e+00,1.000000e-07,80.581516
2,3,1.000000e-07,5.555556,7.407407e+00,1.851852e+00,1.000000e-07,85.185185
3,4,1.492537e+00,5.970149,8.955224e+00,1.492537e+00,1.000000e-07,81.343284
4,5,1.025641e+01,12.820513,5.128205e+00,1.000000e-07,1.000000e-07,71.794872
...,...,...,...,...,...,...,...
62,63,1.829268e+00,9.146341,7.317073e+00,1.219512e+00,1.000000e-07,79.878049
63,64,2.380952e+00,7.142857,2.380952e+00,1.000000e-07,1.000000e-07,88.095238
64,65,2.787456e+00,8.013937,4.529617e+00,1.393728e+00,1.045296e+00,82.229965
65,66,4.545455e+00,13.636364,1.000000e-07,1.000000e-07,1.000000e-07,81.818182


#### Create Chi-Square Statistics for Each:

In [32]:
from scipy.stats import chi2_contingency
location_id=[]
ps=[]

for i in range(0,67):
    location_id.append(i+1)
    x = Employee_Race.loc[i,['Asian','Black or African American','Hispanic or Latino','Native American','Pacific Islander','White, not Hispanic or Latino']].tolist()
    y = Counties_RACE.loc[i,['Asian','Black or African American','Hispanic or Latino','Native American','Pacific Islander','White, not Hispanic or Latino']].tolist()
    data = [x,y]
    stat, p, dof, expected = chi2_contingency(data)
    ps.append(p)

    
Employee_Chi = pd.DataFrame()   
Employee_Chi['Location_id'] = location_id
Employee_Chi['p-Values'] = ps
Employee_Chi.to_csv('Employee_Chi.csv', index=False)
Employee_Chi

,Location_id,p-Values
0,1,0.332306
1,2,0.367902
2,3,0.022200
3,4,0.172886
4,5,0.000024
...,...,...
62,63,0.159316
63,64,0.477928
64,65,0.149644
65,66,0.002711


## Offense
Below, are three fact or reference tables that will be use to build the actual cases per category

### Offense_Criminal

In [36]:
Offense_Criminal = data("Offense_Criminal")
Offense_Criminal = Offense_Criminal.rename(columns={'Offense_ID':'Criminal_ID', 'Offense':'Crime'})
Offense_Criminal.to_csv("Offense_Criminal.csv", index=False)
Offense_Criminal

,Criminal_ID,Crime,Category,Criminal_Class,Type,Fee
0,C1,Possession of Illicit Substance,Drug Offense,Felony,Criminal,250
1,C2,Felony DWI,DWI,Felony,Criminal,500
2,C3,Burglary,Property Crime,Felony,Criminal,1000
3,C4,Larceny,Property Crime,Felony,Criminal,250
4,C5,Theft,Property Crime,Felony,Criminal,500
5,C6,Grand Theft Auto,Property Crime,Felony,Criminal,1000
6,C7,Grand Larceny,Property Crime,Felony,Criminal,250
7,C8,Assault,Violent Crime,Felony,Criminal,500
8,C9,Disorderly,Violent Crime,Felony,Criminal,1000
9,C10,Murder,Violent Crime,Felony,Criminal,250


### Offense_Violations

In [37]:
Offense_Violations = data("Offense_Violations")
Offense_Violations.to_csv("Offense_Violations.csv",index=False)
Offense_Violations

,Violation_ID,Violation,Catgory,Type,Fee
0,V1,Driving while impaired,Misdemeanor,Violation,50
1,V2,Reckless driving,Misdemeanor,Violation,125
2,V3,Driving without a license,Misdemeanor,Violation,75
3,V4,Driving without insurance,Misdemeanor,Violation,300
4,V5,Driving under the influence,Misdemeanor,Violation,100
5,V6,Cell phone violations,Misdemeanor,Violation,50
6,V7,Negligent driving leading to a casualty,Misdemeanor,Violation,125
7,V8,Driving without wearing a safety belt,Traffic Infraction,Violation,75
8,V9,Driving without the use of a proper child rest...,Traffic Infraction,Violation,300
9,V10,Failing to yield the right-of-way,Traffic Infraction,Violation,100


### Offense_Civil

In [38]:
Offense_Civil = data("Offense_Civil")
Offense_Civil = Offense_Civil.rename(columns={'Case_ID':'Civil_ID','Case_Type':'Civil_Type'})
Offense_Civil.to_csv("Offense_Civil.csv",index=False)
Offense_Civil

,Civil_ID,Civil_Type,Category,Type,Fee
0,L1,Motor Vehicle,Tort,Civil,500
1,L2,Nuissance,Tort,Civil,400
2,L3,Intentional,Tort,Civil,250
3,L4,Malicious Prosecution,Tort,Civil,100
4,L5,Slander,Tort,Civil,500
5,L6,Premise Liability,Tort,Civil,400
6,L7,Mass Tort,Mass Tort,Civil,250
7,L8,Landlord/Tenant Dispute,Real Property,Civil,100
8,L9,Mortgage Foreclosure,Real Property,Civil,500
9,L10,Ground Rent,Real Property,Civil,400


## Cases

### Civil_Dispositions
Civil court contains dispositions that are different than Criminal of Traffic(Violations) Courts

In [39]:
Civil_Dispositions = data("Civil_Dispositions")
Civil_Dispositions.to_csv("Civil_Dispositions.csv",index=False)
Civil_Dispositions

,Disposition_ID,Disposition
0,1,Settlements
1,2,Default Judgements
2,3,Dismissed
3,4,Admin Pruge
4,5,Other
5,6,Dispositive Motions
6,7,Arbitration Board
7,8,Withdrawn
8,9,Inactive
9,10,Non-Jury Trial


### Non_Civil_Dispositions

In [40]:
Non_Civil_Dispositions = data("Non_Civil_Dispositions")
Non_Civil_Dispositions.to_csv("Criminal_Dispositions.csv", index=False)
Non_Civil_Dispositions

,Disposition_ID,Disposition
0,1,Dismissed
1,2,Guilty Please
2,3,Non Jury Trial
3,4,Jury Trial
4,5,Inactive
5,6,Nolo Contendre


### Cases 2018 - 2022
Cases are a combination of an Offense, Disposition', 'Location', and 'Date'

#### Civil_Cases (n=1000000)

In [67]:
random.seed(38)
Civil_Cases =  pd.DataFrame()
Civil_Cases['Location_ID'] = milli + ['55'] * 2
Civil_Cases['Case_ID'] =  [*range(1,1000001)]
weights = [100,80,40,20,20,10,10,10,10,4,4,4,4,2,1]
Civil_Cases['Civil_ID'] = random.choices(Offense_Civil.Civil_ID.tolist(), weights=weights, k = 1000000)
Civil_Cases['Disposition_ID'] = random.choices(Civil_Dispositions.Disposition_ID.tolist(), k = 1000000)
years = pd.date_range(start='1/1/2018', end='12/31/2022').date.tolist()
Civil_Cases['Date'] = random.choices(years, k = 1000000)

Data Tweak:
- Increase Bribery in Dauphin County

In [68]:
random.seed(38)
Civil_Cases2 =  pd.DataFrame()
Civil_Cases2['Location_ID'] = ['22']*20000
Civil_Cases2['Case_ID'] =  [*range(6720002,6740002)]
Civil_Cases2['Civil_ID'] = random.choices(['L14'], k = 20000)
Civil_Cases2['Disposition_ID'] = random.choices(Civil_Dispositions.Disposition_ID.tolist(), k = 20000)
years = pd.date_range(start='1/1/2018', end='12/31/2022').date.tolist()
Civil_Cases2['Date'] = random.choices(years, k = 20000)
Civil_Cases = Civil_Cases.append(Civil_Cases2, ignore_index=True)
Civil_Cases.to_csv('Civil_Cases.csv',index = False)
Civil_Cases

C:\Users\rcbre\AppData\Local\Temp\ipykernel_13628\366338120.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Civil_Cases = Civil_Cases.append(Civil_Cases2, ignore_index=True)


,Location_ID,Case_ID,Civil_ID,Disposition_ID,Date
0,1,1,L3,7,2018-08-29
1,1,2,L2,10,2018-05-06
2,1,3,L4,9,2020-06-03
3,1,4,L1,8,2021-02-15
4,1,5,L4,7,2018-11-17
...,...,...,...,...,...
1019995,22,6739997,L14,2,2020-12-15
1019996,22,6739998,L14,4,2019-11-12
1019997,22,6739999,L14,10,2022-04-27
1019998,22,6740000,L14,6,2021-11-09


In [77]:
len(Civil_Cases)

1020000

#### Criminal Cases (n=500000)

In [69]:
random.seed(38)
Criminal_Cases =  pd.DataFrame()
Criminal_Cases['Location_ID'] = half + ['55'] *2
Criminal_Cases['Case_ID'] =  [*range(1000001,1500001)]
weights = [100,80,40,20,20,10,10,10,10,4,4,4,4,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]
Criminal_Cases['Criminal_ID'] = random.choices(Offense_Criminal.Criminal_ID.tolist(), weights=weights, k = 500000)
Criminal_Cases['Disposition_ID'] = random.choices(Non_Civil_Dispositions.Disposition_ID.tolist(), k = 500000)
years = pd.date_range(start='1/1/2018', end='12/31/2022')
Criminal_Cases['Date'] = random.choices(years, k = 500000)

Data Tweak:
- Increase in robberies in Elk County

In [70]:
Criminal_Cases2 = pd.DataFrame()
Criminal_Cases2['Location_ID'] = ['24']*20000
Criminal_Cases2['Case_ID'] =  [*range(6700002,6720002)]
Criminal_Cases2['Criminal_ID'] = random.choices(['C13','C4'],weights=[90,5], k = 20000)
Criminal_Cases2['Disposition_ID'] = random.choices(Non_Civil_Dispositions.Disposition_ID.tolist(), k = 20000)
years = pd.date_range(start='1/1/2018', end='12/31/2022')
Criminal_Cases2['Date'] = random.choices(years, k = 20000)
Criminal_Cases = Criminal_Cases.append(Criminal_Cases2, ignore_index=True)
Criminal_Cases.to_csv('Criminal_Cases.csv',index = False)
Criminal_Cases

C:\Users\rcbre\AppData\Local\Temp\ipykernel_13628\1732313486.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Criminal_Cases = Criminal_Cases.append(Criminal_Cases2, ignore_index=True)


,Location_ID,Case_ID,Criminal_ID,Disposition_ID,Date
0,1,1000001,C3,3,2021-01-17
1,1,1000002,C2,1,2022-05-23
2,1,1000003,C5,3,2021-10-12
3,1,1000004,C1,2,2021-03-09
4,1,1000005,C4,6,2021-01-12
...,...,...,...,...,...
519995,24,6719997,C13,1,2019-01-23
519996,24,6719998,C13,3,2019-06-04
519997,24,6719999,C13,5,2022-07-28
519998,24,6720000,C13,2,2020-11-22


In [76]:
len(Criminal_Cases)

520000

#### Violation Cases (n=5000000)

In [74]:
Violation_Cases =  pd.DataFrame()
Violation_Cases['Location_ID'] = five[1:]
Violation_Cases['Case_ID'] =  [*range(1500001,6500002)][:-1]
weights = [100,80,40,20,20,10,10,10,10,4,4,4,4,2,1,1,1,1,1,1]
Violation_Cases['Violation_ID'] = random.choices(Offense_Violations.Violation_ID.tolist(), weights=weights, k = 5000000)
Violation_Cases['Disposition_ID'] = random.choices([1,2,3], k = 5000000)
years = pd.date_range(start='1/1/2018', end='12/31/2022')
Violation_Cases['Date'] = random.choices(years, k = 5000000)

Data Tweak
- Increase Speeding (V14) at location_id(12,14,18)

In [75]:
Violation_Cases2 = pd.DataFrame()
Violation_Cases2['Location_ID'] = random.choices([2,3,10], k = 200000)
Violation_Cases2['Case_ID'] =  [*range(6500002,6700002)]
Violation_Cases2['Violation_ID'] = random.choices(['V14','V13'],weights=[90,5], k = 200000)
Violation_Cases2['Disposition_ID'] = random.choices([1,2,3], k = 200000)
years = pd.date_range(start='1/1/2018', end='12/31/2022')
Violation_Cases2['Date'] = random.choices(years, k = 200000)
Violation_Cases = Violation_Cases.append(Violation_Cases2, ignore_index=True)
Violation_Cases.to_csv('Violation_Cases.csv', index=False)
Violation_Cases

C:\Users\rcbre\AppData\Local\Temp\ipykernel_13628\3592890281.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Violation_Cases = Violation_Cases.append(Violation_Cases2, ignore_index=True)


,Location_ID,Case_ID,Violation_ID,Disposition_ID,Date
0,1,1500001,V4,1,2019-08-04
1,1,1500002,V4,3,2019-09-20
2,1,1500003,V2,2,2018-11-16
3,1,1500004,V2,2,2022-07-06
4,1,1500005,V2,2,2020-08-29
...,...,...,...,...,...
5199995,2,6699997,V14,3,2021-06-20
5199996,3,6699998,V14,2,2019-07-11
5199997,10,6699999,V14,3,2019-04-12
5199998,10,6700000,V14,1,2018-12-23


#### Speed_Camera_Incidents
- Speed cameras are not documented as a case per se since only a fee is paid online or mailed in
- Speed camera tickets do not add points to PA_DOT record and can be appealed by mailed in evidence
- Speed cameras are being "trialed" in Philadelphia County only since 2019

In [59]:
Philly19 = pd.DataFrame()
Philly19['Location_ID'] = ['51'] * 150000
Philly19['Incident_ID'] = [f'I{i}' for i in range(150000)]
Philly19['Violation_ID'] = ['V20'] * 150000
Philly19['Disposition_ID'] = random.choices(['6', '1'], weights=[9,1], k=150000)
years19 = pd.date_range(start='1/1/2019', end='12/31/2019')
Philly19['Date'] = random.choices(years19, k=150000)

Philly20 = pd.DataFrame()
Philly20['Location_ID'] = ['51'] * 180000
Philly20['Incident_ID'] = [f'I{i}' for i in range(150000,330000)]
Philly20['Violation_ID'] = ['V20'] * 180000
Philly20['Disposition_ID'] = random.choices(['6', '1'], weights=[9,1], k=180000)
years20 = pd.date_range(start='1/1/2020', end='12/31/2020')
Philly20['Date'] = random.choices(years20, k=180000)

Philly21 = pd.DataFrame()
Philly21['Location_ID'] = ['51'] * 70000
Philly21['Incident_ID'] = [f'I{i}' for i in range(330000,400000)]
Philly21['Violation_ID'] = ['V20'] * 70000
Philly21['Disposition_ID'] = random.choices(['6', '1'], weights=[9,1], k=70000)
years21 = pd.date_range(start='1/1/2021', end='12/31/2021')
Philly21['Date'] = random.choices(years21, k=70000)

Philly22 = pd.DataFrame()
Philly22['Location_ID'] = ['51'] * 30000
Philly22['Incident_ID'] = [f'I{i}' for i in range(400000, 430000)]
Philly22['Violation_ID'] = ['V20'] * 30000
Philly22['Disposition_ID'] = random.choices(['6', '1'], weights=[9,1], k=30000)
years22 = pd.date_range(start='1/1/2022', end='12/31/2022')
Philly22['Date'] = random.choices(years22, k=30000)
Philly_Speeding = pd.concat([Philly19,Philly20,Philly21,Philly22], ignore_index=True)
Philly_Speeding.to_csv('Philly_Speeding.csv',index = False)
Philly_Speeding

,Location_ID,Incident_ID,Violation_ID,Disposition_ID,Date
0,51,I0,V20,1,2019-12-17
1,51,I1,V20,6,2019-01-29
2,51,I2,V20,1,2019-09-06
3,51,I3,V20,6,2019-12-17
4,51,I4,V20,6,2019-02-12
...,...,...,...,...,...
429995,51,I429995,V20,6,2022-10-07
429996,51,I429996,V20,6,2022-09-25
429997,51,I429997,V20,6,2022-08-09
429998,51,I429998,V20,1,2022-02-13


## Budgeting 2018 - 2022

### Budget_Funding
Courts receive funding from part of the State budget, the Federal Govt, Fees, and Taxes of course!

In [60]:
Budget_Funding = pd.DataFrame()
Budget_Funding['Years'] = ['2018','2019','2020','2021','2022']
Budget_Funding['Federal'] = [2500000,2800000,3000000,2000000,4000000]
Budget_Funding['State'] = [25000000,2800000,29000000,22000000,24000000]
#Budget_Funding['Fees'] = [#aggregate fees for all three case tables by year]
Budget_Funding['Taxes'] = [2500000,2800000,2500000,2500000,2500000]
Budget_Funding.to_csv('Budge_Funding.csv',index = False)
Budget_Funding

,Years,Federal,State,Taxes
0,2018,2500000,25000000,2500000
1,2019,2800000,2800000,2800000
2,2020,3000000,29000000,2500000
3,2021,2000000,22000000,2500000
4,2022,4000000,24000000,2500000


### Budget_Expenditure
- Overhead, Paying for Facility Operations, and Loss of Fees in certian circumstances.
- In Tableau we can group by 'Year' then 'Location_ID' to get pay per year by location, ect

In [61]:
Budget_Expenditure = pd.DataFrame()
Budget_Expenditure['Years']  = ['2018','2019','2020','2021','2022']
#Budget_Expenditure['Overhead']  = Employee_Data['Salary_2018'].sum() + Employee_Data['Salary_2019'].sum()
#Budget_Expenditure['Facility_Cost']  = [
Budget_Expenditure.to_csv("Budget_Expenditure.csv",index = False)
Budget_Expenditure

,Years
0,2018
1,2019
2,2020
3,2021
4,2022


## Jurors

In [62]:
Jurors = pd.DataFrame()
a = Criminal_Cases[Criminal_Cases['Disposition_ID']==4]['Case_ID'].to_list()
b = Civil_Cases[Civil_Cases['Disposition_ID']==11]['Case_ID'].to_list() 
Jurors['Case_ID'] = a + b
length = len(a+b)
Jurors['Juror_ID'] = [*range(1,length+1)]
races = ['Black or African American','Native American','Asian','Pacific Islander',
'Hispanic or Latino','White, not Hispanic or Latino']
Jurors['Race'] = random.choices(races,weights=[8,1,2,0.5,5.5,83], k = length)
Jurors['Gender'] = random.choices(['M','F'],weights=[49,51], k = length)
languages =['English', 'Spanish', 'Mandarin', 'American Sign', 'Portuguese', 'French', 'Russian']
Jurors['Primary_Language'] = random.choices(languages, weights=[8,2,1,1,1,0.5,0.5],k = length)
Jurors.to_csv('Jurors.csv',index = False)
Jurors

,Case_ID,Juror_ID,Race,Gender,Primary_Language
0,1000015,1,"White, not Hispanic or Latino",M,English
1,1000023,2,"White, not Hispanic or Latino",F,Portuguese
2,1000029,3,Black or African American,F,English
3,1000035,4,"White, not Hispanic or Latino",M,Spanish
4,1000041,5,"White, not Hispanic or Latino",M,Portuguese
...,...,...,...,...,...
179204,6739916,179205,"White, not Hispanic or Latino",F,Spanish
179205,6739935,179206,"White, not Hispanic or Latino",M,Russian
179206,6739948,179207,"White, not Hispanic or Latino",F,English
179207,6739964,179208,Black or African American,M,Portuguese


#### Clean-up Jurors Table To Obtain Racial Makeup for each Group
- Merge on Case_ID with Cases to Get Location_IDs
- Obtained Location_IDs Used to Compare with U.S. Census Location_IDs to subsequently compare racial Proportions

First Get All the Case_ID and Location_ID for each Cases Table from Aboce

In [63]:
Criminal = Criminal_Cases.loc[:,['Location_ID','Case_ID']]
Civil = Criminal_Cases.loc[:,['Location_ID','Case_ID']]
Jurors_Criminal = Jurors.merge(Criminal)
Jurors_Criminal_Civl = Jurors_Criminal.merge(Civil)
Jurors_Race = Jurors_Criminal_Civl.loc[:,['Location_ID','Race']] 
Jurors_Race = Jurors_Race.groupby(['Location_ID'])['Race'].value_counts()
Jurors_Race = Jurors_Race.to_frame()
Jurors_Race = Jurors_Race.rename(columns={'Race':'Count'})
Jurors_Race = Jurors_Race.reset_index()
Jurors_Race = Jurors_Race.pivot(index='Location_ID', columns='Race', values='Count')
Jurors_Race = Jurors_Race.reset_index().rename(index={'Race':'index'})
Jurors_Race = Jurors_Race.fillna(0.0001)
Jurors_Race['Total'] = Jurors_Race.drop(['Location_ID'], axis=1).sum(axis=1)

for columns in Jurors_Race.columns[1:-1]:
    Jurors_Race[f'{columns}'] = np.multiply(np.divide(Jurors_Race[f'{columns}'],Jurors_Race['Total']), 100)
Jurors_Race = Jurors_Race.drop(['Total'], axis=1)
Jurors_Race.Location_ID = Jurors_Race.Location_ID.apply(int)
Jurors_Race = Jurors_Race.sort_values('Location_ID')
Jurors_Race.index = Jurors_Race.index.sort_values()
Jurors_Race.to_csv('Jurors_Race.csv', index=False)
Jurors_Race

Race,Location_ID,Asian,Black or African American,Hispanic or Latino,Native American,Pacific Islander,"White, not Hispanic or Latino"
0,1,2.235469,8.941876,4.769001,0.894188,0.000015,83.159451
1,2,2.009676,8.497705,5.135839,1.004838,0.669892,82.682049
2,3,1.301518,4.338395,6.290672,0.216920,0.433839,87.418655
3,4,1.535682,8.672087,5.600723,0.361337,0.090334,83.739837
4,5,1.223241,8.868499,5.504585,1.834862,0.000031,82.568782
...,...,...,...,...,...,...,...
62,63,2.070552,7.285276,4.371166,1.303681,0.306748,84.662577
63,64,3.466666,10.399997,5.599999,0.000027,0.266667,80.266645
64,65,1.737691,6.826645,5.295821,0.951593,0.289615,84.898635
65,66,0.909090,8.636360,4.999998,1.363636,0.000045,84.090871


#### Chi-Square Test for Racial Proportions for Jurors in PA Counties

In [64]:
from scipy.stats import chi2_contingency
location_id=[]
ps=[]

for i in range(0,67):
    location_id.append(i+1)
    x = Jurors_Race.loc[i,['Asian','Black or African American','Hispanic or Latino','Native American','Pacific Islander','White, not Hispanic or Latino']].tolist()
    y = Counties_RACE.loc[i,['Asian','Black or African American','Hispanic or Latino','Native American','Pacific Islander','White, not Hispanic or Latino']].tolist()
    data = [x,y]
    stat, p, dof, expected = chi2_contingency(data)
    ps.append(p)

    
Jurors_Chi = pd.DataFrame()   
Jurors_Chi['Location_id'] = location_id
Jurors_Chi['p-Values'] = ps
Jurors_Chi.to_csv('Jurors_Chi.csv', index=False)
Jurors_Chi

,Location_id,p-Values
0,1,0.152062
1,2,0.435633
2,3,0.056067
3,4,0.540233
4,5,0.008131
...,...,...
62,63,0.483332
63,64,0.192725
64,65,0.329724
65,66,0.090462
